In [ ]:
import sys
from datetime import datetime

import streamlit as st
import streamlit.components.v1 as components

In [ ]:
# Establish an active Snowflake session
session = get_active_session()

# Use the specified database and schema
session.use_database("MY_DB")
session.use_schema("MY_SCHEMA")

# Define the stage for file operations
stage = '@MY_STAGE'

In [ ]:
# Define the mapping for command-line argument to manager and product details
product_mapping = {
    'a': ('Manager A', 'Widget A', 'WIDGET_A', True),
    'b': ('Manager B', 'Widget B', 'WIDGET_B', True),
    'c': ('Manager C', 'Widget C', 'WIDGET_C', True),
    'test': ('Test Manager', 'Widget A', 'WIDGET_A', False)
}

# Get the command-line argument and ensure it's valid (either 'a', 'b', or 'c')
arg = sys.argv[0] if sys.argv[0] in product_mapping else 'test'

# Use the mapping to set the manager, product, db_product, and prod variables
manager_name, product, db_product, prod = product_mapping[arg]

# Optional: Print the values for debugging
print(f"Manager: {manager_name}, Product: {product}, DB Product: {db_product}, prod: {prod}")

# Get the current date for the report name
today = datetime.now()
current_date = today.strftime('%Y-%m-%d')

# Get report name
report_name = f"{current_date}_{db_product.lower()}_report"

In [ ]:
url_query = f"""
    SELECT GET_PRESIGNED_URL('{stage}', '{db_product.lower()}_reports/{report_name}.pdf', 3600) AS presigned_url
"""

# Execute the query through Snowpark
df = session.sql(url_query)

# # Collect and print the result
result = df.collect()
presigned_url = result[0]['PRESIGNED_URL']

In [ ]:
html = f"""
<!DOCTYPE html>
<html>
<head>
    <title>Daily Sales Report</title>
    <style>
        body {{
            font-family: Arial, sans-serif;
            line-height: 1.6;
            color: #333;
            margin: 0;
            padding: 0;
            background-color: #f5f5f5;
        }}
        .email-container {{
            padding: 20px;
            max-width: 600px;
            margin: auto;
            border: 1px solid #ddd;
            border-radius: 8px;
            background-color: #ffffff;
        }}
        .button {{
            display: inline-block;
            padding: 10px 20px;
            margin-top: 20px;
            font-size: 16px;
            color: #ffffff !important; /* Ensure text color is white */
            background-color: #00949d !important; /* Button background color */
            text-decoration: none;
            border-radius: 5px;
        }}
        .button:hover {{
            background-color: #007a80 !important; /* Darker background on hover */
        }}
        .button:link,
        .button:visited {{
            color: #ffffff !important; /* Enforce white text for all link states */
            background-color: #00949d !important; /* Keep original background color */
            text-decoration: none; /* Remove underline */
        }}
        .button:active {{
            background-color: #006466 !important; /* Darker background on active state */
        }}
        .header {{
            font-size: 18px;
            font-weight: bold;
            color: #00949d;
        }}
    </style>
</head>
<body>
    <div class="email-container">
        <p class="header">Hello {manager_name},</p>
        <p>Here is your daily {product} sales report. Please click the link below to download the report.</p>
        <a href="{presigned_url}" class="button">Download Report</a>
        <p>Reach out to your data team if you have any questions.</p>
    </div>
<div class="footer">
    <body style="margin: 0; padding: 0; font-family: Arial, sans-serif; margin-top: 80px">
        <table width="6%" cellspacing="0" cellpadding="0" style="background-color: #008b8b; padding: 15px 0; border-radius: 12px; overflow: hidden;">
            <tr>
                <td align="center">
                    <table cellspacing="0" cellpadding="0" style="text-align: center; color: white;">
                        <tr>
                            <td style="font-size: 18px; font-weight: bold; padding-bottom: 8px;">
                                WidgetCo
                            </td>
                        </tr>
                        <tr>
                            <td align="center">
                                <!-- Square Logo Shape -->
                                <table cellspacing="2" cellpadding="0" style="border-spacing: 3px; align: center">
                                    <tr>
                                        <td style="background-color: white; width: 8px; height: 8px; border-radius: 2px;"></td>
                                        <td style="background-color: white; width: 8px; height: 8px; border-radius: 2px;"></td>
                                        <td style="background-color: white; width: 8px; height: 8px; border-radius: 2px;"></td>
                                    </tr>
                                    <tr>
                                        <td style="background-color: white; width: 8px; height: 8px; border-radius: 2px;"></td>
                                        <td style="background-color: white; width: 8px; height: 8px; border-radius: 2px;"></td>
                                        <td style="background-color: white; width: 8px; height: 8px; border-radius: 2px;"></td>
                                    </tr>
                                    <tr>
                                        <td style="background-color: white; width: 8px; height: 8px; border-radius: 2px;"></td>
                                        <td style="background-color: white; width: 8px; height: 8px; border-radius: 2px;"></td>
                                        <td style="background-color: white; width: 8px; height: 8px; border-radius: 2px;"></td>
                                    </tr>
                                </table>
                            </td>
                        </tr>
                    </table>
                </td>
            </tr>
        </table>
        <p>From your friendly WidgetCo data team.</p>
    </body>
</div>

</body>
</html>
"""

In [ ]:
subject_line = f'{current_date} {product} sales report'

In [ ]:
if prod == False:
    st.markdown(subject_line)
    components.html(html)

In [ ]:
send_email_sql = f"""
CALL SYSTEM$SEND_EMAIL(
    'report_email_integration',
    'me@email.com',
    '{subject_line}',
    '{html}',
    'text/html'
    )
"""

In [ ]:
if prod == True:
    session.sql(send_email_sql).collect()